Task 1 - Install Spark, load required libraries, set environment variables, initiate Spark, load file

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz


The system cannot find the path specified.
tar: Error opening archive: Failed to open 'spark-2.4.6-bin-hadoop2.7.tgz'


In [8]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [9]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

NameError: name 'spark' is not defined

In [ ]:
# !pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/openjdk-20_windows-x64_bin/jdk-20"
os.environ["SPARK_HOME"] = "/spark-3.4.0-bin-hadoop3/spark-3.4.0-bin-hadoop3"
import findspark
findspark.init()
# from google.colab import files
# files.upload()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder
print("JAVA_HOME: ", os.environ['JAVA_HOME'])
print("SPARK_HOME: ", os.environ['SPARK_HOME'])
# print("HADOOP_HOME: ", os.environ['HADOOP_HOME'])

In [ ]:
spark=SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
!dir

In [ ]:
data=spark.read.csv('data.csv',inferSchema=True, header=True)

In [ ]:
data.printSchema()

In [ ]:
data.describe().toPandas().transpose()

In [ ]:
def replace(column, value):
    return when(column != value,column).otherwise(lit(None))

In [ ]:
data = data.withColumn('Market Category', replace(col('Market Category'), 'N/A'))

In [ ]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

In [ ]:
data = data.drop('Market Category')
data = data.na.drop()

In [ ]:
print((data.count(),len(data.columns)))

In [ ]:
assembler = VectorAssembler(inputCols=['Year','Engine HP','Engine Cylinder','Number of Doors','highway MPG','city mpg','Popularity'],
                            outputCol='Attributes')
regressor = RandomForestRegressor(featuresCol='Attributes', labelCol='MSRP')
pipeline = Pipeline(stages=[assembler,regressor])
pipeline.write().overwrite().save('pipeline')